In [20]:
# Problem Statement:
# Given a partial potential knights tour, can we QUICKLY fill it out?
# 16 squares will be labeled with the position
import numpy as np
from numpy import uint8, int8
from itertools import permutations, combinations
from random import randint
from numba import njit, prange
import time

In [2]:
C = 300540195 #31 choose 15, = 3, 3, 5, 17, 19, 23, 29, 31
P = 1307674368000 #15! = 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 5, 5, 5, 7, 7, 11, 13 
# a good block size would be 8000x3, 32000x9, 8000x51, 8000x153
#given these two numbers, choose 16 of the 32 blocks, and name them accordingly

def fact(n):
    return 1 if n < 1 else n * fact(n-1)

def choose(n, k):
    return fact(n)//(fact(k) * fact(n-k)) if n >= k else 0

'''https://en.wikipedia.org/wiki/Combinatorial_number_system'''
def enumerate_comb(c, n, k):
    #c = enumeration, n = total number, k = how many chosen
    l = []
    for den in range(k, 0, -1):
#         print(den)
        #count down from k
        for num in range(0, n+1, 1):
#             print("\t{}".format(num), end=": ")
            #count up the c1...ck
            ch = choose(n-num, den)
#             print(ch)
            if ch <= c:
                l.append(num - 1) #0 based indexing
                c -= choose(n-num, den)
                break;
    return tuple(l)

def enumerate_perm(p, n):
    nums = []
    for i in range(n, 0, -1):
        number_of_nodes_in_layer = i #how many items are in the layer we are on
        size_of_node_in_layer = fact(i-1)
        nums.append(p//size_of_node_in_layer)
        p = p%size_of_node_in_layer
    vals = list(range(n))
    permutation = []
    for n in nums:
        permutation.append(vals.pop(n))
#     print(nums)
#     print(vals)
#     print(permutation)
#     print()
    return tuple(permutation)

In [3]:
def get_rand_init(log=True):
    prand = randint(0,P-1)
    crand = randint(0,C-1)
    p = enumerate_perm(prand, 15)
    c = enumerate_comb(crand, 31, 15)
    if log:
        print("perm: {}, comb: {}".format(prand,crand))
    for j in range(15):
        assert j in p
    for c_item in c:
        assert c_item < 31 and c_item >= 0
        assert c.count(c_item) == 1
    return gen_board_from_pc(p,c)

def gen_board_from_pc(perm, comb):
    board = np.ones(64, dtype=int8) * -1
    board[0] = 0;
    black_squares = [2,4,6,9,11,13,15,16,18,20,22,25,27,29,31,32,32,36,38,41,43,45,47,48,50,52,54,57,59,61,63]
    for idx in range(15):
        board[black_squares[comb[idx]]] = (perm[idx]+1)*4
    return board

@njit
def print_board(board):
    chars = ['0','1','2','3','4','5','6','7','8','9']
    print('------------------------')
    for i in range(8):
        print_str = ''
        for j in range(8):
            num = board[i*8+j]
            if num != -1:
                print_str += (chars[num//10] if num//10 != 0 else ' ')
                print_str += chars[num%10]
                print_str += ' '
            else:
                print_str += '   '
        print(print_str)
    print('------------------------')

In [4]:
print_board(get_rand_init())

perm: 433023053207, comb: 249292128
------------------------
 0    20    60          
         36     4       
      12          40    
   32           8       
44                24    
   28    56             
52                48    
                     16 
------------------------


In [5]:
kng = np.array([
    [17, 10, -1, -1, -1, -1, -1, -1],[18, 11, 16, -1, -1, -1, -1, -1],
    [19, 12,  8, 17, -1, -1, -1, -1],[20, 13,  9, 18, -1, -1, -1, -1],
    [21, 14, 10, 19, -1, -1, -1, -1],[22, 15, 11, 20, -1, -1, -1, -1],
    [23, 12, 21, -1, -1, -1, -1, -1],[13, 22, -1, -1, -1, -1, -1, -1],
    [25, 18,  2, -1, -1, -1, -1, -1],[26, 19,  3, 24, -1, -1, -1, -1],
    [27, 20,  4,  0, 16, 25, -1, -1],[28, 21,  5,  1, 17, 26, -1, -1],
    [29, 22,  6,  2, 18, 27, -1, -1],[30, 23,  7,  3, 19, 28, -1, -1],
    [31,  4, 20, 29, -1, -1, -1, -1],[ 5, 21, 30, -1, -1, -1, -1, -1],
    [33, 26, 10,  1, -1, -1, -1, -1],[34, 27, 11,  2,  0, 32, -1, -1],
    [35, 28, 12,  3,  1,  8, 24, 33],[36, 29, 13,  4,  2,  9, 25, 34],
    [37, 30, 14,  5,  3, 10, 26, 35],[38, 31, 15,  6,  4, 11, 27, 36],
    [39,  7,  5, 12, 28, 37, -1, -1],[ 6, 13, 29, 38, -1, -1, -1, -1],
    [41, 34, 18,  9, -1, -1, -1, -1],[42, 35, 19, 10,  8, 40, -1, -1],
    [43, 36, 20, 11,  9, 16, 32, 41],[44, 37, 21, 12, 10, 17, 33, 42],
    [45, 38, 22, 13, 11, 18, 34, 43],[46, 39, 23, 14, 12, 19, 35, 44],
    [47, 15, 13, 20, 36, 45, -1, -1],[14, 21, 37, 46, -1, -1, -1, -1],
    [49, 42, 26, 17, -1, -1, -1, -1],[50, 43, 27, 18, 16, 48, -1, -1],
    [51, 44, 28, 19, 17, 24, 40, 49],[52, 45, 29, 20, 18, 25, 41, 50],
    [53, 46, 30, 21, 19, 26, 42, 51],[54, 47, 31, 22, 20, 27, 43, 52],
    [55, 23, 21, 28, 44, 53, -1, -1],[22, 29, 45, 54, -1, -1, -1, -1],
    [57, 50, 34, 25, -1, -1, -1, -1],[58, 51, 35, 26, 24, 56, -1, -1],
    [59, 52, 36, 27, 25, 32, 48, 57],[60, 53, 37, 28, 26, 33, 49, 58],
    [61, 54, 38, 29, 27, 34, 50, 59],[62, 55, 39, 30, 28, 35, 51, 60],
    [63, 31, 29, 36, 52, 61, -1, -1],[30, 37, 53, 62, -1, -1, -1, -1],
    [58, 42, 33, -1, -1, -1, -1, -1],[59, 43, 34, 32, -1, -1, -1, -1],
    [60, 44, 35, 33, 40, 56, -1, -1],[61, 45, 36, 34, 41, 57, -1, -1],
    [62, 46, 37, 35, 42, 58, -1, -1],[63, 47, 38, 36, 43, 59, -1, -1],
    [39, 37, 44, 60, -1, -1, -1, -1],[38, 45, 61, -1, -1, -1, -1, -1],
    [50, 41, -1, -1, -1, -1, -1, -1],[51, 42, 40, -1, -1, -1, -1, -1],
    [52, 43, 41, 48, -1, -1, -1, -1],[53, 44, 42, 49, -1, -1, -1, -1],
    [54, 45, 43, 50, -1, -1, -1, -1],[55, 46, 44, 51, -1, -1, -1, -1],
    [47, 45, 52, -1, -1, -1, -1, -1],[46, 53, -1, -1, -1, -1, -1, -1]
], dtype=int8)

degree = np.array([
    2,3,4,4,4,4,3,2,
    3,4,6,6,6,6,4,3,
    4,6,8,8,8,8,6,4,
    4,6,8,8,8,8,6,4,
    4,6,8,8,8,8,6,4,
    4,6,8,8,8,8,6,4,
    3,4,6,6,6,6,4,3,
    2,3,4,4,4,4,3,2
], dtype=int8)

order = np.array([0,4,8,12,16,20,24,28,32,36,40,44,48,52,56,60,0], dtype=int8)

translate = np.array([
	'@','0','1','2','3','4','5','6','7','8','9','a','b','c','d','e',
	'f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u',
	'v','w','x','y','z','+','A','B','C','D','E','F','G','H','I','J',
	'K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
])

# visited = np.array([
#     -1,-1,-1,-1,-1,-1,-1,-1,
#     -1,-1,-1,-1,-1,-1,-1,-1,
#     -1,-1,-1,-1,-1,-1,-1,-1,
#     -1,-1,-1,-1,-1,-1,-1,-1,
#     -1,-1,-1,-1,-1,-1,-1,-1,
#     -1,-1,-1,-1,-1,-1,-1,-1,
#     -1,-1,-1,-1,-1,-1,-1,-1,
#     -1,-1,-1,-1,-1,-1,-1,-1
# ], dtype=int8)

In [6]:
# Test that the enumerate_<perm|comb> methods are working as intended, and degree/kng are consistent
perms = []
for i in range(fact(7)):
    perms.append(enumerate_perm(i, 7))
combs = []
for i in range(choose(14,6)):
    combs.append(enumerate_comb(i, 14, 6))

lp = list(permutations(range(7))) 
lc = list(combinations(range(14),6))

for i in perms:
    assert i in lp
for i in lp:
    assert i in perms
for i in combs:
    assert i in lc
for i in lc:
    assert i in combs

for i in range(64):
    v = 8 - degree[i] - np.count_nonzero(kng[i] == -1)
    assert v == 0

In [7]:
@njit
def fill(idx, visited):
    #draw a path from order[idx] to order[idx+1]
    if idx == 16:
        print("CLOSED TOUR:")
        print_board(visited)
        return
        
    start_number = order[idx]
    end_number = order[idx+1]
    s1 = -1
    s5 = -1
    for i in range(64):
        if visited[i] == start_number:
            s1 = i
        if visited[i] == end_number:
            s5 = i
    
    ds1 = degree[s1]
    ds5 = degree[s5]
    
    for i in range(ds1):
        s2 = kng[s1][i]
        if visited[s2] == -1:
            for j in range(ds5):
                s4 = kng[s5][j]
                if visited[s4] == -1 and s2 != s4:
                    ds2 = degree[s2]
                    for k in range(ds2):
                        s3 = kng[s2][k]
                        if visited[s3] == -1:
                            s3_in_kng_s4 = False
                            for l in range(8):
                                if kng[s4][l] == s3:
                                    s3_in_kng_s4 = True
                                    break;
                            if s3_in_kng_s4 and s3 != s1 and s3 != s2 and s3 != s4 and s3 != s5:
                                visited[s2] = start_number + 1
                                visited[s3] = start_number + 2
                                visited[s4] = start_number + 3
#                                     print_board(visited)
#                                     print("{} -> {} -> {} -> {} -> {}".format(s1,s2,s3,s4,s5))
                                fill(idx+1, visited)
                                visited[s2] = -1
                                visited[s3] = -1
                                visited[s4] = -1
    return


In [8]:
# 20:14:25 is when the 10,000 run started: 20:17:46 is when it ended (3.5 mins), 4,000,000 per day per core
# 4e6 per core/day, 
#1.25 x 10^22 total
# = 3e15 core/days for all

In [9]:
l = [0,37,30,51,16,55,8,53,29,32,1,36,9,52,15,56,38,63,34,31,50,17,54,7,33,28,49,2,35,10,57,14,24,39,62,43,18,13,6,11,27,44,25,48,3,60,19,58,40,23,46,61,42,21,12,5,45,26,41,22,47,4,59,20]
for i in range(len(l)):
    if l[i] not in order:
        l[i] = -1
ll = np.array(l)
ll
print_board(ll)
fill(0, ll)

------------------------
 0          16     8    
   32    36    52    56 
                        
   28                   
24                      
   44    48    60       
40                12    
                4    20 
------------------------
CLOSED TOUR:
------------------------
 0 37 30 51 16 55  8 53 
29 32  1 36  9 52 15 56 
38 63 34 31 50 17 54  7 
33 28 49  2 35 10 57 14 
24 39 62 43 18 13  6 11 
27 44 25 48  3 60 19 58 
40 23 46 61 42 21 12  5 
45 26 41 22 47  4 59 20 
------------------------
CLOSED TOUR:
------------------------
 0 35 30 53 16 55  8 51 
29 32  1 36  9 52 15 56 
38 63 34 31 54 17 50  7 
33 28 37  2 49 10 57 14 
24 39 62 43 18 13  6 11 
27 44 25 48  3 60 19 58 
40 23 46 61 42 21 12  5 
45 26 41 22 47  4 59 20 
------------------------


In [ ]:
# visited = gen_board_from_pc(enumerate_perm(199098, 16), enumerate_comb(17898061, 32, 16))
for i in range(10000000): 
    print(i, end='\r')
    visited = get_rand_init(log=False)
#     print_board(visited)
    fill(0, visited)

In [28]:
# @njit(parallel=True)
def dist_run(ip,ic): #1000x51 blocks -> ~10-15 minute runtime
    for dp in range(1000):
        print(dp, end="\r")
        in_perm = enumerate_perm(ip+dp, 15)
        for dc in range(51):
            in_comb = enumerate_comb(ic+dc, 31, 15)
            in_board = gen_board_from_pc(in_perm, in_comb)
            fill(0, in_board)

In [ ]:
start_time = time.time()
dist_run(1,1)
end_time = time.time()
print()
print(int(end_time-start_time))